# Tutorial

In [23]:
import ccp

Q_ = ccp.Q_
%config Completer.use_jedi = False

`ccp` uses pint to handle units. Q_ is a pint quantity.

Here lets define a suction pressure `ps` and a suction temperature `Ts`.

For the suction pressure we are going to use pint.

In [24]:
ps = Q_(3, "bar")
Ts = 300

The pint objects hold the magnitude and units for a variable and can be used for unit conversion:

In [25]:
print(
    f"ps: {ps}\n"
    f"ps magnitude: {ps.magnitude}\n"
    f"ps units: {ps.units}\n"
    f"convert to atm: {ps.to('atm')}\n"
)

ps: 3 bar
ps magnitude: 3
ps units: bar
convert to atm: 2.960769800148039 standard_atmosphere



We define the fluid composition is defined as a dictionary:

In [26]:
fluid = {
    "CarbonDioxide": 0.79585,
    "R134a": 0.16751,
    "Nitrogen": 0.02903,
    "Oxygen": 0.00761,
}

We can create the suction and discharge states using the `State` class:

In [27]:
suc0 = ccp.State.define(fluid=fluid, p=ps, T=Ts)
disch0 = ccp.State.define(fluid=fluid, p=Q_(7.255, "bar"), T=391.1)

Notice that in the cell above we defined the states using pint quantities mixed with pure floats.

The way that `ccp` works is by assuming that float are in the SI units system, so 

```python
disch0 = ccp.State.define(fluid=fluid, p=Q_(7.255, 'bar'), T=391.1)
```

is the same as 

```python
disch0 = ccp.State.define(fluid=fluid, p=Q_(7.255, 'bar'), T=Q_(391.1, 'degK'))
```

Create performance point(s):

In [28]:
point0 = ccp.Point(
    suc=suc0,
    disch=disch0,
    speed=Q_(7941, "RPM"),
    flow_m=Q_(34203.6, "kg/hr"),
    b=0.0285,
    D=0.365,
)
point1 = ccp.Point(
    suc=suc0,
    disch=disch0,
    speed=Q_(7941, "RPM"),
    flow_m=Q_(37203.6, "kg/hr"),
    b=0.0285,
    D=0.365,
)

Now we can create an impeller, which is basically a container for points.

In [29]:
imp = ccp.Impeller([point0, point1])

In [30]:
imp

To show other ccp features we are going to load an impeller from `csv` files created with the [Engauge Digitizer](https://markummitchell.github.io/engauge-digitizer/) application.
For more information on how to use ccp with Engaguge Digitizer see {ref}`this How-to Guide <engauge>`

In [32]:
imp = ccp.impeller.impeller_example()

The impeller object stores all points in the `.points` attribute. 

It is also possible to get an specific point in the performance map using the `.point()` method:

In [39]:
imp.point(flow_v=5.5, speed=900)

Point(suc=State.define(p=Q_("408000 Pa"), T=Q_("307 K"), fluid={"METHANE": 0.58976, "CO2": 0.36605, "ETHANE": 0.03099, "PROPANE": 0.00600, "NITROGEN": 0.00550, "BUTANE": 0.00080, "ISOBUTAN": 0.00050, "H2S": 0.00020, "PENTANE": 0.00010, "IPENTANE": 0.00010}), speed=Q_("900 rad/s"), flow_v=Q_("5.50 m³/s"), head=Q_("103450 J/kg"), eff=Q_("0.748"))

We can also plot performance parameters such as the polytropic head. 
In this case we can choose a flow and speed and the correspondent curve and point will also be plotted:

In [40]:
fig = imp.head_plot(flow_v=5.5, speed=900)
fig

It is also possible to plot discharge parameters such as T (temperature), p (pressure), rho (specific mass) and so on:

In [41]:
fig = imp.disch.T_plot(flow_v=5.5, speed=900)
fig

In this plots we can also choose the units used for plotting:

In [42]:
fig = imp.disch.rho_plot(
    flow_v=5.5, speed=900, flow_v_units="m³/h", speed_units="RPM", rho_units="g/cm³"
)
fig

Notice that by default the units used in the `flow_v` and `speed` arguments are the SI units, but you can also provide pint quantities:

In [43]:
fig = imp.disch.rho_plot(
    flow_v=Q_(20000, "m³/h"),
    speed=Q_(8594, "RPM"),
    flow_v_units="m³/h",
    speed_units="RPM",
    rho_units="g/cm³",
)
fig

In [44]:
fig.data

(Scatter({
     'name': '6882.0 RPM',
     'x': array([11218.7       , 11356.63103448, 11494.56206897, 11632.49310345,
                 11770.42413793, 11908.35517241, 12046.2862069 , 12184.21724138,
                 12322.14827586, 12460.07931034, 12598.01034483, 12735.94137931,
                 12873.87241379, 13011.80344828, 13149.73448276, 13287.66551724,
                 13425.59655172, 13563.52758621, 13701.45862069, 13839.38965517,
                 13977.32068966, 14115.25172414, 14253.18275862, 14391.1137931 ,
                 14529.04482759, 14666.97586207, 14804.90689655, 14942.83793103,
                 15080.76896552, 15218.7       ]),
     'y': array([0.00780144, 0.00778184, 0.00776044, 0.00773737, 0.0077128 , 0.00768686,
                 0.0076597 , 0.00763148, 0.00760234, 0.00757243, 0.00754189, 0.0075108 ,
                 0.00747914, 0.00744686, 0.00741393, 0.00738029, 0.00734559, 0.00730922,
                 0.00727054, 0.00722889, 0.00718364, 0.007134  , 0.00707912, 